In [10]:
#1
import sys
import nltk
import sklearn
import numpy
import pandas

print('Python: {}'.format(sys.version))
print('NLTK: {}'.format(nltk.__version__))
print('Scikit-learn: {}'.format(sklearn.__version__))
print('Pandas: {}'.format(pandas.__version__))
print('Numpy: {}'.format(numpy.__version__))

Python: 3.7.1 (default, Dec 10 2018, 22:54:23) [MSC v.1915 64 bit (AMD64)]
NLTK: 3.4
Scikit-learn: 0.20.1
Pandas: 0.23.4
Numpy: 1.15.4


## 1. Load the Dataset

In [11]:
#2
import pandas as pd
import numpy as np
# load the dataset of sms messages
df=pd.read_table('SMSSpamCollection',header=None , encoding='utf-8')


In [12]:
#3
# print useful information about data set
print(df.info())
print(df.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
0    5572 non-null object
1    5572 non-null object
dtypes: object(2)
memory usage: 87.1+ KB
None
      0                                                  1
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [13]:
#4
# check class distribution
classes=df[0]
print(classes.value_counts())


ham     4825
spam     747
Name: 0, dtype: int64


## 2. Preprocess the Data

In [14]:
#5
# convert class labels to binary values , 0=ham , 1=spam
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
Y = encoder.fit_transform(classes)
print(classes[:10])
print(Y[:10])


0     ham
1     ham
2    spam
3     ham
4     ham
5    spam
6     ham
7     ham
8    spam
9    spam
Name: 0, dtype: object
[0 0 1 0 0 1 0 0 1 1]


In [15]:
#6
# store the SMS message data
text_messages=df[1]
print(text_messages[:10])

0    Go until jurong point, crazy.. Available only ...
1                        Ok lar... Joking wif u oni...
2    Free entry in 2 a wkly comp to win FA Cup fina...
3    U dun say so early hor... U c already then say...
4    Nah I don't think he goes to usf, he lives aro...
5    FreeMsg Hey there darling it's been 3 week's n...
6    Even my brother is not like to speak with me. ...
7    As per your request 'Melle Melle (Oru Minnamin...
8    WINNER!! As a valued network customer you have...
9    Had your mobile 11 months or more? U R entitle...
Name: 1, dtype: object


In [16]:
#7
# use regular expression to replace email address , phone numbers , url, other number, symbols
# replace email address with 'emailaddr'
processed = text_messages.str.replace(r'^.+@[^\.].*\.[a-z]{2,}$','emailaddr')

# replace urls with 'webaddress'
processed = processed.str.replace(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$','webaddress')

# replace money symbols with 'moneysymb'
processed = processed.str.replace(r'£|\$','moneysymb')

# replace 10 digit phone number with 'phonenumber'
processed = processed.str.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$','phonenumber')

# replace normal number with 'numbr'
processed = processed.str.replace(r'\d+(\.\d+)?','numbr')

In [17]:
#8
# remove punctuation
processed = processed.str.replace(r'[^\w\d\s]',' ')

# replace whitespace between terms with a single space
processed = processed.str.replace(r'\s+',' ')

# remove leading and trailing whitespace
processed = processed.str.replace(r'^\s+|\s+?$', '')

In [18]:
#9
# change words to lowercase - Hello,HELLO,hello are all same words
processed = processed.str.lower()
print(processed)

0       go until jurong point crazy available only in ...
1                                 ok lar joking wif u oni
2       free entry in numbr a wkly comp to win fa cup ...
3             u dun say so early hor u c already then say
4       nah i don t think he goes to usf he lives arou...
5       freemsg hey there darling it s been numbr week...
6       even my brother is not like to speak with me t...
7       as per your request melle melle oru minnaminun...
8       winner as a valued network customer you have b...
9       had your mobile numbr months or more u r entit...
10      i m gonna be home soon and i don t want to tal...
11      six chances to win cash from numbr to numbr nu...
12      urgent you have won a numbr week free membersh...
13      i ve been searching for the right words to tha...
14                      i have a date on sunday with will
15      xxxmobilemovieclub to use your credit click th...
16                                 oh k i m watching here
17      eh u r

In [19]:
#10
# remove stop words from text messages
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
processed = processed.apply(lambda x: ' '.join(term for term in x.split() if term not in stop_words))

In [20]:
#11
# remove word stem using porter stemmer
ps = nltk.PorterStemmer()

processed = processed.apply(lambda x: ' '.join(ps.stem(term) for term in x.split()))

In [21]:
#12
print(processed)

0       go jurong point crazi avail bugi n great world...
1                                   ok lar joke wif u oni
2       free entri numbr wkli comp win fa cup final tk...
3                     u dun say earli hor u c alreadi say
4                    nah think goe usf live around though
5       freemsg hey darl numbr week word back like fun...
6           even brother like speak treat like aid patent
7       per request mell mell oru minnaminungint nurun...
8       winner valu network custom select receivea mon...
9       mobil numbr month u r entitl updat latest colo...
10      gonna home soon want talk stuff anymor tonight...
11      six chanc win cash numbr numbr numbr pound txt...
12      urgent numbr week free membership moneysymbnum...
13      search right word thank breather promis wont t...
14                                            date sunday
15      xxxmobilemovieclub use credit click wap link n...
16                                             oh k watch
17      eh u r

In [22]:
#13
from nltk.tokenize import word_tokenize

# creating a bag-of-words
all_words = []

for message in processed:
    words = word_tokenize(message)
    for w in words:
        all_words.append(w)
        
all_words = nltk.FreqDist(all_words)

In [23]:
#14
# print total number words and 15 most common words
print('number of words: {}'.format(len(all_words)))
print('most common words: {}'.format(all_words.most_common(15)))

number of words: 6579
most common words: [('numbr', 2648), ('u', 1207), ('call', 674), ('go', 456), ('get', 451), ('ur', 391), ('gt', 318), ('lt', 316), ('come', 304), ('moneysymbnumbr', 303), ('ok', 293), ('free', 284), ('day', 276), ('know', 275), ('love', 266)]


In [24]:
#15
# use 1500 most common words as features
word_features = list(all_words.keys())[:1500]

In [25]:
#16
# define a find_features function

def find_features(message):
    words = word_tokenize(message)
    features = {}
    for word in word_features:
        features[word] = (word in words)
        
    return features

# lets see an example
features = find_features(processed[0])
for key,value in features.items():
    if value == True:
        print(key)

go
jurong
point
crazi
avail
bugi
n
great
world
la
e
buffet
cine
got
amor
wat


In [26]:
#17
# find features for all messages
messages = list(zip(processed , Y))

# define a seed for reproducibility
seed = 1
np.random.seed = seed
np.random.shuffle(messages)

# call find_features function for each SMS messages
featuresets = [(find_features(text), label) for (text,label) in messages]

In [27]:
#18
# split training and testion data sets using sklearn
from sklearn import model_selection
training, testing = model_selection.train_test_split(featuresets, test_size = 0.25 , random_state = seed)

In [28]:
#19
print('training : {}'.format(len(training)))
print('testing : {}'.format(len(testing)))

training : 4179
testing : 1393


## 3. scikit-learn classifiers with NLTK

In [29]:
#20
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression , SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report , accuracy_score , confusion_matrix

In [30]:
#21
# define models to train
names = ['K Nearest Neighbors','Decision Tree','Random Forest','Logistic Regression','SGD Classifier','Naive Bayes','SVM Linear']

#initialize our classifiers
classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticRegression(),
    SGDClassifier(max_iter = 100),
    MultinomialNB(),
    SVC(kernel = 'linear')
]

models = zip(names,classifiers)


In [31]:
#22
# wrap models in nltk
from nltk.classify.scikitlearn import SklearnClassifier

for name,model in models:
    nltk_model = SklearnClassifier(model)
    nltk_model.train(training)
    accuracy = nltk.classify.accuracy(nltk_model,testing)*100
    print('{}: Accuracy: {}'.format(name , accuracy))

K Nearest Neighbors: Accuracy: 94.18521177315147
Decision Tree: Accuracy: 97.48743718592965


C:\Users\Kunal\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Random Forest: Accuracy: 98.63603732950466


C:\Users\Kunal\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Regression: Accuracy: 98.34888729361091


C:\Users\Kunal\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


SGD Classifier: Accuracy: 98.06173725771716
Naive Bayes: Accuracy: 98.56424982053123
SVM Linear: Accuracy: 98.42067480258436


In [35]:
#23
# ensemble method - voting classifier

from sklearn.ensemble import RandomForestClassifier, VotingClassifier

# define models to train
names = ['K Nearest Neighbors','Decision Tree','Random Forest','Logistic Regression','SGD Classifier','Naive Bayes','SVM Linear']

#initialize our classifiers
classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticRegression(),
    SGDClassifier(max_iter = 100),
    MultinomialNB(),
    SVC(kernel = 'linear')
]

models = list(zip(names,classifiers))

nltk_ensemble = SklearnClassifier(VotingClassifier(estimators = models, voting = 'hard', n_jobs = -1))
nltk_ensemble.train(training)
accuracy = nltk.classify.accuracy(nltk_ensemble,testing)*100
print('Ensemble Accuracy: {}'.format(accuracy))

Ensemble Accuracy: 98.42067480258436


In [36]:
#24
# make class label prediction for testing set
txt_featuers , labels = zip(*testing)
prediction = nltk_ensemble.classify_many(txt_featuers)

In [42]:
#25
# print a confusion matrix and classification report
print(classification_report(labels , prediction))
pd.DataFrame(
    confusion_matrix(labels , prediction),
    index = [['actual' , 'actual'] , ['ham' , 'spam']],
    columns = [['predicted' , 'predicted'] , ['ham' , 'spam']]
)

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1204
           1       0.99      0.89      0.94       189

   micro avg       0.98      0.98      0.98      1393
   macro avg       0.99      0.95      0.96      1393
weighted avg       0.98      0.98      0.98      1393



predicted     
                  ham spam
actual ham       1202    2
       spam        20  169